In [0]:
%sql
SELECT * FROM json.`/Volumes/gizmobox/landing/operations_volume/customers/`;

In [0]:
%sql
use catalog gizmobox;
use schema bronze;
CREATE OR REPLACE VIEW v_customer
AS
SELECT * FROM json.`/Volumes/gizmobox/landing/operations_volume/customers/`;

In [0]:
%sql
SELECT max(created_timestamp) max_timestamp, customer_id 
    FROM v_customer WHERE customer_id IS NOT NULL GROUP BY customer_id ;

- ### Removing Duplicates

In [0]:
%sql
with actual_data as 
    (SELECT DISTINCT * FROM v_customer WHERE customer_id IS NOT NULL),
    max_timestamp as
    (SELECT max(created_timestamp) as max_timestamp, customer_id 
    FROM v_customer WHERE customer_id IS NOT NULL GROUP BY customer_id )
SELECT a.*
    FROM actual_data a
    JOIN max_timestamp t
    ON a.customer_id = t.customer_id
    AND a.created_timestamp = t.max_timestamp ORDER BY a.customer_id;

- ### Casting to correct datatypes

In [0]:
%sql
with actual_data as 
    (SELECT DISTINCT * FROM v_customer WHERE customer_id IS NOT NULL),
    max_timestamp as
    (SELECT max(created_timestamp) as max_timestamp, customer_id 
    FROM v_customer WHERE customer_id IS NOT NULL GROUP BY customer_id )
SELECT CAST(a.created_timestamp AS TIMESTAMP) AS created_timestamp,
       a.customer_id,
       a.customer_name,
       CAST(a.date_of_birth AS DATE) AS date_of_birth,
       a.email,
       CAST(a.member_since AS DATE) AS member_since,
       a.telephone
    FROM actual_data a
    JOIN max_timestamp t
    ON a.customer_id = t.customer_id
    AND a.created_timestamp = t.max_timestamp ORDER BY a.customer_id;

- ### Writing to Delta table

In [0]:
%sql
CREATE TABLE gizmobox.silver.customer
AS
with actual_data as 
    (SELECT DISTINCT * FROM v_customer WHERE customer_id IS NOT NULL),
    max_timestamp as
    (SELECT max(created_timestamp) as max_timestamp, customer_id 
    FROM v_customer WHERE customer_id IS NOT NULL GROUP BY customer_id )
SELECT CAST(a.created_timestamp AS TIMESTAMP) AS created_timestamp,
       a.customer_id,
       a.customer_name,
       CAST(a.date_of_birth AS DATE) AS date_of_birth,
       a.email,
       CAST(a.member_since AS DATE) AS member_since,
       a.telephone
    FROM actual_data a
    JOIN max_timestamp t
    ON a.customer_id = t.customer_id
    AND a.created_timestamp = t.max_timestamp ORDER BY a.customer_id;

In [0]:
%sql
SELECT * FROM gizmobox.silver.customer

In [0]:
%sql
DESCRIBE EXTENDED gizmobox.silver.customer;